## Aggregate Parallel Texts in directory

This assumes that we have a bunch of ```.csv``` files with the filename in the format ```${source}-${lang}.csv```, where each file has the header ```ts,txt``` to read in the text at each numeric timestamp.

In [ ]:
import os
import csv
import time, random
import re

In [ ]:
lang_from, lang_to = 'en', 'en'

data_path = './data'

Go through all the files in the directory, and find the ```source``` prefixes that have both ```lang_from``` and ```lang_to``` CSVs available.

In [ ]:
stub_from, stub_to = set(),set()
stub_matcher = re.compile(r"(.*)\-(\w+)\.csv")
for fname in os.listdir(data_path):
    #print(fname)
    m = stub_matcher.match(fname)
    if m:
        stub, lang = m.group(1), m.group(2)
        if lang == lang_from: stub_from.add(stub)
        if lang == lang_to:   stub_to.add(stub)
stub_both = stub_from.intersection(stub_to)

Now, go through ```stub_both``` and for each CSVs, read in both languages, and take all the ```txt``` entries at the same timestamps, and build the correspondence.

In [ ]:
correspondence_loc,txt_from,txt_to=[],[],[]

def read_dict_from_csv(fname):
    d=dict()
    with open(fname, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            d[float(row['ts'])]=row['txt']
    return d

for stub in stub_both:
    #print("Reading stub %s" % (stub,))
    data_from = read_dict_from_csv( os.path.join(data_path, stub+'-'+lang_from+'.csv') )
    data_to   = read_dict_from_csv( os.path.join(data_path, stub+'-'+lang_to+'.csv') )
    
    valid, skipped=0, 0
    for ts, txt in data_from.items():
        if ts in data_to:
            correspondence_loc.append( (stub, ts) )
            txt_from.append( txt )
            txt_to.append( data_to[ts] )
            valid += 1
        else:
            skipped += 1
    print("%3d valid of %3d fragments from '%s'" % (valid, valid+skipped, stub))
print("  Total data : %d text fragments" % (len(correspondence_loc),)) 

In [ ]:
for _ in range(10):
    i = random.randrange(len(correspondence_loc))
    print( txt_from[i], txt_to[i]  )

### Tokenize the correspondences
NB: Japanese requires word-splitting too

In [ ]:
if lang_from=='ja' or lang_to=='ja':
    import tinysegmenter
    ja_segmenter = tinysegmenter.TinySegmenter()

sub_punctuation = re.compile(r'[\,\.\:\;\?\!\-\s\"0-9\(\)]+')
sub_apostrophes = re.compile(r'\'(\w+)')
sub_multispaces = re.compile(r'\s\s+')
    
def tokenize_txt(arr, lang):
    tok=[]
    for txt in arr:
        t = re.sub(sub_punctuation, u' ', txt.lower())
        if "'" in t:
            t = re.sub(sub_apostrophes, r" '\1", t)
        if lang=='ja':
            t = ' '.join( ja_segmenter.tokenize(t) )
        t = re.sub(sub_multispaces, ' ', t)
        tok.append(t.strip())
    return tok

In [ ]:
tok_from = tokenize_txt(txt_from, lang_from)
tok_to   = tokenize_txt(txt_to, lang_to)

In [ ]:
tok_from[220:250]

In [ ]:
tok_to[220:250]

### Build frequency dictionaries

### Build a fragment coindicedence matrix
This might allow us to do single word translations...